In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import pandas as pd
import tensorflow as tf


# Import and clean df

In [2]:
# Import cleaned VAERS dataset
vaers_df = pd.read_csv('VAERSDATA.csv', low_memory=False)
vaers_df.head()

,Unnamed: 0,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,...,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,...,NaN,NaN,NaN,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,...,NaN,NaN,NaN,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
2,2,916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",...,23.1,NaN,NaN,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,3,916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",...,NaN,NaN,NaN,COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
4,4,916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",...,23.1,NaN,NaN,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))


In [3]:
# Check vaers_df column types
vaers_df.dtypes

Unnamed: 0           int64
VAERS_ID             int64
RECVDATE            object
STATE               object
AGE_YRS            float64
CAGE_YR            float64
CAGE_MO            float64
SEX                 object
RPT_DATE            object
SYMPTOM_TEXT        object
DIED                object
DATEDIED            object
L_THREAT            object
ER_VISIT            object
HOSPITAL            object
HOSPDAYS           float64
X_STAY              object
DISABLE             object
RECOVD              object
VAX_DATE            object
ONSET_DATE          object
NUMDAYS            float64
LAB_DATA            object
V_ADMINBY           object
V_FUNDBY            object
OTHER_MEDS          object
CUR_ILL             object
HISTORY             object
PRIOR_VAX           object
SPLTTYPE            object
FORM_VERS            int64
TODAYS_DATE         object
BIRTH_DEFECT        object
OFC_VISIT           object
ER_ED_VISIT         object
ALLERGIES           object
SYMPTOM1            object
S

In [4]:
# change dates(objects)columns to datetime
#vaers_df['RECVDATE'] = pd.to_datetime(vaers_df['RECVDATE'])
#vaers_df['RPT_DATE'] = pd.to_datetime(vaers_df['RPT_DATE'])
#vaers_df['DATEDIED'] = pd.to_datetime(vaers_df['DATEDIED'])
#vaers_df['VAX_DATE'] = pd.to_datetime(vaers_df['VAX_DATE'])
#vaers_df['ONSET_DATE'] = pd.to_datetime(vaers_df['ONSET_DATE'])
#vaers_df['TODAYS_DATE'] = pd.to_datetime(vaers_df['TODAYS_DATE'])

#vaers_df.dtypes

In [5]:
# Categorical variable list
vaers_cat = vaers_df.dtypes[vaers_df.dtypes == 'object'].index.tolist()
vaers_cat

['RECVDATE',
 'STATE',
 'SEX',
 'RPT_DATE',
 'SYMPTOM_TEXT',
 'DIED',
 'DATEDIED',
 'L_THREAT',
 'ER_VISIT',
 'HOSPITAL',
 'X_STAY',
 'DISABLE',
 'RECOVD',
 'VAX_DATE',
 'ONSET_DATE',
 'LAB_DATA',
 'V_ADMINBY',
 'V_FUNDBY',
 'OTHER_MEDS',
 'CUR_ILL',
 'HISTORY',
 'PRIOR_VAX',
 'SPLTTYPE',
 'TODAYS_DATE',
 'BIRTH_DEFECT',
 'OFC_VISIT',
 'ER_ED_VISIT',
 'ALLERGIES',
 'SYMPTOM1',
 'SYMPTOM2',
 'SYMPTOM3',
 'SYMPTOM4',
 'SYMPTOM5',
 'VAX_TYPE',
 'VAX_MANU',
 'VAX_LOT',
 'VAX_DOSE_SERIES',
 'VAX_ROUTE',
 'VAX_SITE',
 'VAX_NAME']

In [6]:
# Unique values in each column
vaers_df[vaers_cat].nunique()

RECVDATE              70
STATE                 60
SEX                    3
RPT_DATE              20
SYMPTOM_TEXT       27195
DIED                   1
DATEDIED              95
L_THREAT               1
ER_VISIT               1
HOSPITAL               1
X_STAY                 1
DISABLE                1
RECOVD                 3
VAX_DATE             278
ONSET_DATE           206
LAB_DATA            7726
V_ADMINBY              9
V_FUNDBY               4
OTHER_MEDS         11816
CUR_ILL             3518
HISTORY             9515
PRIOR_VAX           1033
SPLTTYPE            5895
TODAYS_DATE          104
BIRTH_DEFECT           1
OFC_VISIT              1
ER_ED_VISIT            1
ALLERGIES           6245
SYMPTOM1            2301
SYMPTOM2            2220
SYMPTOM3            2017
SYMPTOM4            1833
SYMPTOM5            1617
VAX_TYPE              40
VAX_MANU              14
VAX_LOT             1887
VAX_DOSE_SERIES        8
VAX_ROUTE              7
VAX_SITE               9
VAX_NAME              63


In [7]:
# DIED value counts
counts = vaers_df.DIED.value_counts()
counts

Y    2575
Name: DIED, dtype: int64

In [8]:
# LAB DATA value counts
counts = vaers_df.LAB_DATA.value_counts()
counts

None                                                                                              2684
none                                                                                              1557
no                                                                                                 222
NONE                                                                                               192
None.                                                                                              171
                                                                                                  ... 
Transported to Emergency Room for evaluation  and released                                           1
Test Date: 202101; Test Name: tested; Result Unstructured Data: Test Result:positive for COVID       1
03/03/2021 blood tests show nothing of consequence                                                   1
Multiple x-rays, ct scans and ultrasounds at Medical Center while hospita

In [9]:
# OTHER MEDS value counts
counts = vaers_df.OTHER_MEDS.value_counts()
counts

None                                                                                                                                                                                1866
none                                                                                                                                                                                 875
unknown                                                                                                                                                                              360
Unknown                                                                                                                                                                              359
No                                                                                                                                                                                   131
                                                                           

In [10]:
# CUR_ILL value counts
counts = vaers_df.CUR_ILL.value_counts()
counts

None                                 6322
none                                 3455
No                                    654
no                                    522
NONE                                  365
                                     ... 
Controlled high blood pressure,         1
Asthma; Benign brain neoplasm NOS       1
sinus issue                             1
Had Covid 11/15/20 - 12/17/20           1
To be determined                        1
Name: CUR_ILL, Length: 3518, dtype: int64

In [11]:
# HISTORY value counts
counts = vaers_df.HISTORY.value_counts()
counts

None                                                                                                                                                      3153
none                                                                                                                                                      1525
Comments: List of non-encoded Patient Relevant History: Patient Other Relevant History 1: None                                                             676
Medical History/Concurrent Conditions: No adverse event (No reported medical history)                                                                      375
No                                                                                                                                                         339
                                                                                                                                                          ... 
Medical History/Concurrent Conditions: Sinus i

In [12]:
# PRIOR_VAX value counts
counts = vaers_df.PRIOR_VAX.value_counts()
counts

Flu                                                                                  18
unknown                                                                              13
Flu Vaccine                                                                          10
COVID19 (Moderna) on 1/26/2021                                                        9
yes                                                                                   8
                                                                                     ..
Moderna Covid-19 vaccine                                                              1
Pt. states  reaction to flu vaccine                                                   1
SIMILAR EVENT DESCRIBED DUE TO A FLU SHOT PER PATIENT. OTHER INFORMATION UNKNOWN.     1
Zoster vaccine age 55 or so. Red, hot rash, area at injection site                    1
Flu symptoms                                                                          1
Name: PRIOR_VAX, Length: 1033, d

In [13]:
#SPLTTYPE value counts
counts = vaers_df.SPLTTYPE.value_counts()
counts

USMODERNATX, INC.MOD20210    1291
vsafe                         514
VSAFE                          73
USMODERNATX, INC.MOD20200      49
USGLAXOSMITHKLINEUS2020GS      29
                             ... 
USPFIZER INC2021034513          1
USPFIZER INC2021093295          1
USPFIZER INC2020517666          1
USPFIZER INC2021099311          1
USPFIZER INC2020520786          1
Name: SPLTTYPE, Length: 5895, dtype: int64

In [14]:
#ALLERGIES value counts
counts = vaers_df.ALLERGIES.value_counts()
counts

None                            3369
none                            1619
NKA                              804
NKDA                             769
No                               401
                                ... 
Lisinopril, Codeine                1
Latex, Seasonal allergies          1
PENICILLIN MORPHINE BENADRYL       1
Sulfa meds and Phenergan           1
Benzoperoxide                      1
Name: ALLERGIES, Length: 6245, dtype: int64

In [15]:
#SYMPTOM1 value counts
counts = vaers_df.SYMPTOM1.value_counts()
counts

Chills                     2358
Arthralgia                 1516
Headache                   1197
Injection site erythema    1176
Dizziness                  1175
                           ... 
Rectal tenesmus               1
Abdominal adhesions           1
Ventricular drainage          1
Breast discomfort             1
Listless                      1
Name: SYMPTOM1, Length: 2301, dtype: int64

In [16]:
#SYMPTOM2 value counts
counts = vaers_df.SYMPTOM2.value_counts()
counts

Headache                          1683
Fatigue                           1301
Chills                            1118
Dizziness                          835
Pyrexia                            825
                                  ... 
Labile blood pressure                1
Cardiac ventriculogram left          1
Antibody test                        1
Hypertrophy of tongue papillae       1
Herpes zoster meningitis             1
Name: SYMPTOM2, Length: 2220, dtype: int64

In [17]:
#VAX_LOT value counts
counts = vaers_df.VAX_LOT.value_counts()
counts

039K20A    1375
011J20A    1260
EK9231     1132
EK5730     1072
EH9899     1047
           ... 
En6202        1
26L20A        1
Es1686        1
paa           1
028L2OA       1
Name: VAX_LOT, Length: 1887, dtype: int64

In [18]:
# Drop LAB_DATA, OTHER MEDS, CUR_ILL, HISTORY, PRIOR_VAX, ALLERGIES - unless we use NLP, those columns can't be hot encoded
vaers_drop = vaers_df.drop(columns = ['LAB_DATA', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'ALLERGIES'])

In [19]:
# Drop Unamed: ) because it is the same as the index
vaers_clean = vaers_drop.drop(columns = 'Unnamed: 0')

In [20]:
# re-count Unique values in each column in new cleaned df
vaers_clean.nunique()

VAERS_ID           27955
RECVDATE              70
STATE                 60
AGE_YRS              118
CAGE_YR              105
CAGE_MO                7
SEX                    3
RPT_DATE              20
SYMPTOM_TEXT       27195
DIED                   1
DATEDIED              95
L_THREAT               1
ER_VISIT               1
HOSPITAL               1
HOSPDAYS              33
X_STAY                 1
DISABLE                1
RECOVD                 3
VAX_DATE             278
ONSET_DATE           206
NUMDAYS              115
V_ADMINBY              9
V_FUNDBY               4
SPLTTYPE            5895
FORM_VERS              2
TODAYS_DATE          104
BIRTH_DEFECT           1
OFC_VISIT              1
ER_ED_VISIT            1
SYMPTOM1            2301
SYMPTOMVERSION1        2
SYMPTOM2            2220
SYMPTOMVERSION2        2
SYMPTOM3            2017
SYMPTOMVERSION3        2
SYMPTOM4            1833
SYMPTOMVERSION4        2
SYMPTOM5            1617
SYMPTOMVERSION5        2
VAX_TYPE              40


In [21]:
vaers_clean.dtypes

VAERS_ID             int64
RECVDATE            object
STATE               object
AGE_YRS            float64
CAGE_YR            float64
CAGE_MO            float64
SEX                 object
RPT_DATE            object
SYMPTOM_TEXT        object
DIED                object
DATEDIED            object
L_THREAT            object
ER_VISIT            object
HOSPITAL            object
HOSPDAYS           float64
X_STAY              object
DISABLE             object
RECOVD              object
VAX_DATE            object
ONSET_DATE          object
NUMDAYS            float64
V_ADMINBY           object
V_FUNDBY            object
SPLTTYPE            object
FORM_VERS            int64
TODAYS_DATE         object
BIRTH_DEFECT        object
OFC_VISIT           object
ER_ED_VISIT         object
SYMPTOM1            object
SYMPTOMVERSION1    float64
SYMPTOM2            object
SYMPTOMVERSION2    float64
SYMPTOM3            object
SYMPTOMVERSION3    float64
SYMPTOM4            object
SYMPTOMVERSION4    float64
S

# One Hot Encoder

In [49]:
# replace missing data with Simple Inputer
imp = SimpleImputer(strategy='constant', fill_value="None")

vaers_imp = pd.DataFrame(imp.fit_transform(vaers_clean))
vaers_imp.columns = vaers_clean.columns
vaers_imp

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,916600,01/01/2021,TX,33.0,33.0,None,F,None,Right side of epiglottis swelled up and hinder...,None,...,None,None,None,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,916601,01/01/2021,CA,73.0,73.0,None,F,None,Approximately 30 min post vaccination administ...,None,...,None,None,None,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
2,916602,01/01/2021,WA,23.0,23.0,None,F,None,"About 15 minutes after receiving the vaccine, ...",None,...,23.1,None,None,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916603,01/01/2021,WA,58.0,58.0,None,F,None,"extreme fatigue, dizziness,. could not lift my...",None,...,None,None,None,COVID19,MODERNA,unknown,UNK,None,None,COVID19 (COVID19 (MODERNA))
4,916604,01/01/2021,TX,47.0,47.0,None,F,None,"Injection site swelling, redness, warm to the ...",None,...,23.1,None,None,COVID19,MODERNA,None,1,IM,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40256,1092595,03/11/2021,TX,70.0,70.0,None,F,None,A high risk employee received 1st vaccine dose...,Y,...,None,None,None,COVID19,MODERNA,025J20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
40257,1092651,03/11/2021,CA,77.0,77.0,None,M,None,DEATH,Y,...,None,None,None,COVID19,MODERNA,0124214,1,IM,LA,COVID19 (COVID19 (MODERNA))
40258,1092653,03/11/2021,CA,88.0,88.0,None,F,None,Patient received the vaccine on the evening of...,Y,...,None,None,None,COVID19,MODERNA,unknown,2,SYR,AR,COVID19 (COVID19 (MODERNA))
40259,1092737,03/11/2021,ME,69.0,69.0,None,M,None,"Sudden death. Alone at home, found on floor 4 ...",Y,...,None,None,None,COVID19,MODERNA,None,1,UN,AR,COVID19 (COVID19 (MODERNA))


In [50]:
vaers_imp.dtypes

VAERS_ID           object
RECVDATE           object
STATE              object
AGE_YRS            object
CAGE_YR            object
CAGE_MO            object
SEX                object
RPT_DATE           object
SYMPTOM_TEXT       object
DIED               object
DATEDIED           object
L_THREAT           object
ER_VISIT           object
HOSPITAL           object
HOSPDAYS           object
X_STAY             object
DISABLE            object
RECOVD             object
VAX_DATE           object
ONSET_DATE         object
NUMDAYS            object
V_ADMINBY          object
V_FUNDBY           object
SPLTTYPE           object
FORM_VERS          object
TODAYS_DATE        object
BIRTH_DEFECT       object
OFC_VISIT          object
ER_ED_VISIT        object
SYMPTOM1           object
SYMPTOMVERSION1    object
SYMPTOM2           object
SYMPTOMVERSION2    object
SYMPTOM3           object
SYMPTOMVERSION3    object
SYMPTOM4           object
SYMPTOMVERSION4    object
SYMPTOM5           object
SYMPTOMVERSI

In [51]:
# Recriate the vares_clean categorical list
vaers_cat = vaers_imp.dtypes[vaers_imp.dtypes == 'object'].index.tolist()
vaers_cat

['VAERS_ID',
 'RECVDATE',
 'STATE',
 'AGE_YRS',
 'CAGE_YR',
 'CAGE_MO',
 'SEX',
 'RPT_DATE',
 'SYMPTOM_TEXT',
 'DIED',
 'DATEDIED',
 'L_THREAT',
 'ER_VISIT',
 'HOSPITAL',
 'HOSPDAYS',
 'X_STAY',
 'DISABLE',
 'RECOVD',
 'VAX_DATE',
 'ONSET_DATE',
 'NUMDAYS',
 'V_ADMINBY',
 'V_FUNDBY',
 'SPLTTYPE',
 'FORM_VERS',
 'TODAYS_DATE',
 'BIRTH_DEFECT',
 'OFC_VISIT',
 'ER_ED_VISIT',
 'SYMPTOM1',
 'SYMPTOMVERSION1',
 'SYMPTOM2',
 'SYMPTOMVERSION2',
 'SYMPTOM3',
 'SYMPTOMVERSION3',
 'SYMPTOM4',
 'SYMPTOMVERSION4',
 'SYMPTOM5',
 'SYMPTOMVERSION5',
 'VAX_TYPE',
 'VAX_MANU',
 'VAX_LOT',
 'VAX_DOSE_SERIES',
 'VAX_ROUTE',
 'VAX_SITE',
 'VAX_NAME']

In [52]:
# OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

#Fit and transform the OneHotEncoder
vaers_enc_df = pd.DataFrame(enc.fit_transform(vaers_imp[vaers_cat]))

#Add the encoded variable names to the DF
vaers_enc_df.columns = enc.get_feature_names(vaers_cat)
vaers_enc_df.head()

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']

In [26]:
vaers_enc_df

,VAERS_ID_916600,VAERS_ID_916601,VAERS_ID_916602,VAERS_ID_916603,VAERS_ID_916604,VAERS_ID_916606,VAERS_ID_916607,VAERS_ID_916608,VAERS_ID_916609,VAERS_ID_916610,...,VAX_NAME_TDAP (NO BRAND NAME),VAX_NAME_TETANUS TOXOID (NO BRAND NAME),VAX_NAME_TYPHOID LIVE ORAL TY21A (VIVOTIF),VAX_NAME_VACCINE NOT SPECIFIED (NO BRAND NAME),VAX_NAME_VACCINE NOT SPECIFIED (OTHER),VAX_NAME_VARICELLA (VARIVAX),VAX_NAME_YELLOW FEVER (YF-VAX),VAX_NAME_ZOSTER (NO BRAND NAME),VAX_NAME_ZOSTER (SHINGRIX),VAX_NAME_ZOSTER LIVE (ZOSTAVAX)
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# Merge with vaers_clean and drop original cat. list
vaers_df = vaers_clean.merge(vaers_enc_df, left_index=True, right_index=True)
vaers_df = vaers_df.drop(vaers_cat,1)


In [35]:
#Search for columns with DIED in name to find the target column (y)
died_cols = [col for col in vaers_df.columns if 'DIED' in col]
died_cols

['SYMPTOM_TEXT_DEATH Narrative: NO ADDITIONAL DETAIL PROVIDED OTHER THAN PATIENT DIED AT HOME',
 'SYMPTOM_TEXT_DIED',
 'SYMPTOM_TEXT_DIED WITHIN 5 DAYS OF RECEIEVING THE 2ND DOSE, EXPERIENCED GENERALIZED WEAKNESS.',
 'SYMPTOM_TEXT_MY WIFE DIED UNEXPECTEDLY 4 DAYS AFTER HER SECOND DOSAGE SHOT, ON FEBRUARY 17, 2021.  SHE HAD BEEN HEALTHY AND HAD A RECENT CHECKUP AT WHICH THE DOCTOR GAVE  HER A CLEAN BILL OF HEALTH.  SHE WAS ALERT AND IN GOOD SPIRITS JUST THE NIGHT BEFORE WHEN WE WATCHED A MOVIE TOGETHER.  I SAW NO INDICATION THAT SHE WAS FEELING POORLY OR OTHERWISE.  I FOUND HER IN BED, DECEASED, UPON COMING HOME FROM WORK THE NEXT DAY.',
 'SYMPTOM_TEXT_PATIENT DIED IN HIS SLEEP NIGHT AFTER ADMINISTRATION',
 'SYMPTOM_TEXT_PATIENT WAS ADMITTED TO ER FOR  ALTERED MENTAL STATUS  / UTI  SEPSIS  WITH SEPTIC SHOCK / COVID AND COVID   PNA   PATIENT WAS ADMITTED TO ICU AND DIED .   POA WISH TO WITHDRAWL EXTRME MEASURES',
 'SYMPTOM_TEXT_Patient reported to Emergency room on 01/23/2021 with compla

# Split and Scale Data

In [37]:
# Split data into features and targets
y = vaers_df['DIED_Y']
X = vaers_df.drop(columns = 'DIED_Y')

In [44]:
# Counting values on y
y.value_counts()

1.0    40261
Name: DIED_Y, dtype: int64

In [38]:
# Split into test and training
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [39]:
# StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Logistic Regression

## Resampling the data

In [ ]:
# Counting values in Y_train


In [42]:
#Create a Logistic Regression Model
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=78)
classifier

LogisticRegression(max_iter=200, random_state=78)

In [43]:
# Train the data
classifier.fit(X_train, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1.0

In [ ]:
# Evaluate
y_pred = classifier.predict(X_test)
print(f' Logistic regression model accuracy: {accuracy_score(y_test, y_pred):.3f}')